<a href="https://colab.research.google.com/github/sambhavpurohit14/Smart_Gallery/blob/encoder-functions/transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
sentence -> tokens -> inputs ID (location in the vocab) -> embedding
'''

import torch
import torch.nn
import math

In [ ]:
class SentenceEmbeddings(nn.Module):

  def __init__(self, model_dimensions : int, vocab_size : int):
    super().__init__()
    self.model_dimensions = model_dimensions
    self.vocab_size = vocab_size
    self.embedding = nn.Embedding(self.vocab_size, self.model_dimensions)

    '''
    embedding maps the numbers to a vector, and those numbers are learned during training
    paper multiplies the embeddings by sqrt(model_dimensions)
    '''

  def forward(self, inputs):
    return self.embedding(inputs) * math.sqrt(self.model_dimensions)

In [ ]:
class PositionalEncoding(nn.Module):

  def __init__(self, model_dimensions : int, max_length : int, dropout : float) -> None:
    super().__init__()
    self.model_dimensions = model_dimensions
    self.max_length = max_length
    self.dropout = nn.Dropout(dropout)

    '''
    we need vectors of d model size and max_length number
    even positions - sines, odd positions cos
    '''
    # numerical stability
    position_encoding = torch.zeros(max_length, model_dimensions)
    position = torch.arange(0, max_length, dtype=torch.float).unsqueeze(1)
    denominator = torch.exp(torch.arange(0, model_dimensions, 2).float() * (-math.log(10000.0) / model_dimensions))

    position_encoding[:,0::2] = torch.sin(position * demominator)
    position_encoding[:,1::2] = torch.cos(position * denominator)
    '''
    keep in model, but not as a parameter - buffer
    '''
    positional_encoding = position_encoding.unsqueeze(0).transpose(0,1)
    self.register_buffer('positional_encoding', positional_encoding)

  def forward(self, inputs):
    inputs = inputs + (self.positional_encoding[:, :x.shape[1], :]).requires_grad_(False)
    return self.dropout(inputs)




In [ ]:
'''
layer normalization - calculate mean and variance for each item independently of others

'''
class LayerNormalization(nn.Module):

  def __init__(self, epsilon: float = 10** -8) -> None:
    super().__init__()
    self.epsilon = epsilon
    self.w = nn.Parameter(torch.ones(1))
    self.b = nn.Parameter(torch.zeros(1))

  def forward(self, inputs):
    mean = inputs.mean(dim = -1, keepdim = True)
    std = x.std(dim = -1, keepdim = True)
    return self.alpha * (x - mean) / (std + self.epsilon) + self.b

In [ ]:
class FeedForward(nn.Module):

  def __init__(self, model_dimensions : int, ff : int, dropout:float) -> None:
    super().__init__()
    self.linear_1 = nn.Linear(model_dimensions, ff)
    self.dropout = nn.Dropout(dropout)
    self.linear_2 = nn.Linear(ff, model_dimensions)

  def forward(self, inputs):
    # change of dimensions
    return self.linear_2(self.dropout(torch.relu(self.linear_1(inputs))))

In [ ]:
''' multihead attention
define 3 matrices q, k, v
in case of encoder, all are same
W is learned

attention = softmax((QK^T) / sqrt(d_k)
get 1D heads, multiply by Wo
 '''

 class MultiHeadAttention(nn.Module):

  def __init__(self, model_dimensions : int, h:int, dropout : float) -> None:
    super().__init__()
    self.model_dimensions = model_dimensions
    self.h = h
    assert model_dimensions % h == 0, "CHECK Y DIS"
    self.d_k = model_dimensions // h

    self.w_q = nn.Linear(model_dimensions, model_dimensions)
    self.w_k = nn.Linear(model_dimensions, model_dimensions)
    self.w_v = nn.Linear(model_dimensions, model_dimensions)

    self.w_o = nn.Linear(model_dimensions, model_dimensions)
    self.dropout = nn.Dropout(dropout)

  @staticmethod
  def attention(query, key, value, mask, dropout:nn.Dropout):
    d_k = query.shape[-1]
    attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
      attention_scores.masked_fill_(mask == 0, -1e9)





  def forward(self, q, k, v, mask):
    '''
    check dimensions with paper, should not change
        '''
    query = self.w_q(q)
    key = self.w_k(k)
    value = self.w_v(v)
    ''''
    batch,max_len,model_dim -> batch,max_len,h,d_k
    '''
    query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1,2)
    key = query.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1,2)
    value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1,2)

    x, self.attention_scores = MultiHeadAttention.attention(query, key, value, mask, self.dropout)
    # exchange 1 and 2, more swap than transpose
    x = x.transpose(1, 2).contiguos().view(x.shape[0], -1, self.h * self.d_k)
    return self.w_o(x)




In [ ]:
class ResidualConnection(nn.Module):

  def __init__(self, dropout : float) -> None:
    super().__init__()
    self.layer_norm = LayerNormalization()
    self.dropout = nn.Dropout(dropout)

  def forward(self, inputs, sublayer):
    return x + self.dropout(sublayer(self.layer_norm(inputs)))

In [ ]:
class EncoderBlock(nn.Module):

  def __init__(self, self_attention_block: MultiHeadAttention, feed_forward_block: FeedForward, dropout: float) -> None:
    super().__init__()
    self.self_attention_block = self_attention_block
    self.feed_forward_block = feed_forward_block
    self.residual_connections = n.ModuleList([ResidualConnection(dropout) for _ in range(2)])

  def forward(self, inputs, mask):
    x = self.residual_connections[0](inputs, lambda x: self.self_attention_block(x, x, x, mask))
    x = self.residual_connections[1](x, self.feed_forward_bock)
    return x

In [ ]:
class Encoder(nn.Module):

  def __init__(self, layers: n.ModuleList) -> None:
    super().__init__()
    self.layers = layers
    self.norm = LayerNormalization()

    def forward(self, inputs, mask):
    x = inputs
    for layer in self.layers:
      x = layer(x, mask)
    return self.norm(x)